In [46]:
MLP_EPISODES_DIRECTORIES = [
    'D:/Videos/S1', 
    'D:/Videos/S2', 
    'D:/Videos/S3', 
    'D:/Videos/S4', 
    'D:/Videos/S5', 
    'D:/Videos/S6', 
    'D:/Videos/S7', 
    'D:/Videos/S8', 
    'D:/Videos/S9', 
]
EPISODE_EXTENSIONS = ['mp4', 'mkv']
EPISODE_PATTERNS = [r'S(\d{2})E(\d{2})', r'(\d{2})x(\d{2})']

import os
import re

episode_index = {}
for d in MLP_EPISODES_DIRECTORIES:
    for root,_,files in os.walk(d):
        for f in files:
            f_ext = f.split('.')[-1]
            if not f_ext in EPISODE_EXTENSIONS:
                continue
            for pattern in EPISODE_PATTERNS:
                match_s = re.search(pattern, f)
                if match_s:
                    sig = f's{match_s.group(1)}e{match_s.group(2)}'
                    episode_index[sig] = os.path.join(root,f).replace('\\','/')
                
import subprocess
from pydub import AudioSegment
import io
def extract_center_channel(vid_path):
    command = ['ffmpeg', '-i', vid_path, '-filter_complex',
        'channelsplit=channel_layout=5.1:channels=FC[FC]', '-map', '[FC]',
        '-c:a', 'pcm_s32le',
         '-f', 'wav', '-']
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, errs = process.communicate()
    f = io.BytesIO(output)
    audio_segment = AudioSegment.from_file(f, format='wav')
    return audio_segment

from demucs import pretrained
from demucs.apply import apply_model
import numpy as np
from pathlib import Path
import torch
import scipy.io.wavfile
class DemucsModels:
    def __init__(self, repo, models):
        if type(repo) == str:
            repo = Path(repo)
        self.models = []
        self.model_names = []
        for model in models:
            self.models.append(pretrained.get_model(name=model, repo=repo))
            self.model_names.append(model)

    def apply(self, x):
        outputs = []
        for model in self.models:
            out = apply_model(model, torch.from_numpy(x).T.unsqueeze(0),
                device='cuda')[0]
            out_dict = {}
            for name, source in zip(model.sources, out):
                out_dict[name] = source
            vocals = out_dict['vocals'].numpy().T
            outputs.append(vocals)
        return outputs

    def process_ep(self, specifier='s01e01', dump_to_folder=None):
        assert specifier in episode_index
        if dump_to_folder is not None:
            os.makedirs(dump_to_folder, exist_ok=True)

        print(f"processing {specifier}")
        print("extracting center")
        fc = extract_center_channel(episode_index[specifier])
        sr = fc.frame_rate
        fc_dbfs = fc.dBFS
        fc_orig = fc
        fc = np.array(fc.get_array_of_samples())
        fc = np.column_stack((fc,fc))
        fc = (fc / (2 ** 32 - 1)).astype(np.float32)

        print("applying demucs")
        demucs_outputs = self.apply(fc)
        print("finished applying demucs")
        for i,do in enumerate(demucs_outputs):
            do_io = io.BytesIO()
            scipy.io.wavfile.write(do_io, 48000, do)
            do_io.seek(0)
            do_snd = AudioSegment.from_wav(do_io).set_channels(1)
            print(f'dbfs adjust {fc_dbfs - do_snd.dBFS}')
            do_snd = do_snd.apply_gain(fc_dbfs - do_snd.dBFS)
            if dump_to_folder is not None:
                do_snd.export(
                    os.path.join(dump_to_folder,f'{specifier}_demu{i}.flac'), format='flac')

        if dump_to_folder is not None:
            # These are more for previews than anything
            fc_orig.export(
                os.path.join(dump_to_folder,f'{specifier}.flac'),
                format='flac')
        print('done')

demucsmodels = DemucsModels(repo="D:/Code/demucs/release_models",
    models=["SFX_separation_epoch_190", "SFX_separation_v2_epoch_338"])
for k in episode_index.keys():
    dump_folder = os.path.join('episode_process_dumps',k)
    demucsmodels.process_ep(specifier=k, dump_to_folder=dump_folder)


# Objective: Compare 0. original (i.e. Clipper's choice), 1. sfx demucs v1, 2. sfx demucs v2

processing s01e01
extracting center
applying demucs
finished applying demucs
done
processing s01e02
extracting center
applying demucs


KeyboardInterrupt: 